In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars_examples.table.grammar_decoration import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

from pydrake.all import Quaternion, RigidTransform

import meshcat
import meshcat.geometry as meshcat_geom

In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Meshcat url:  http://127.0.0.1:7003/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [3]:
trees_to_draw = []
#with open("target_dataset_examples.pickle", "rb") as f:
#with open("structure_constraint_examples.pickle", "rb") as f:
#with open("pre_fit_grammar_draws.pickle", "rb") as f:
with open("post_fit_grammar_draws.pickle", "rb") as f:
#with open("baseline_post_fit_grammar_draws.pickle", "rb") as f:
    while 1:
        try:
            #trees_to_draw.append(pickle.load(f))
            trees_to_draw.append(pickle.load(f)[0])
        except EOFError:
            break
print("Loaded %d environments" % len(trees_to_draw))
trees_to_draw = trees_to_draw[:30]

Loaded 50 environments


In [36]:
tree = trees_to_draw[0]

# Using the set of decoration mapping rules, create a new set of decoration
# nodes and recursively expand them by following their production rules. Add
# these to scene tree and project the whole thing to feasibility with forward
# sim.
decorated_tree = deepcopy(tree)
# Freeze existing stuff.
for node in decorated_tree.nodes:
    if node.physics_geometry_info is not None:
        node.physics_geometry_info.fixed = True
new_pairs = []
for existing_type, new_type in decoration_mapping.items():
    for existing_node in decorated_tree.nodes:
        if isinstance(existing_node, existing_type):
            new_node = new_type(existing_node.tf)
            new_pairs.append((existing_node, new_node))
expand_queue = []
for existing_node, new_node in new_pairs:
    # This node is *not* connected to its parent, since there is no
    # rule on the parent for explaining this new node.
    decorated_tree.add_node(new_node)
    expand_queue.append(new_node)
# Now do recursive expansion of any nodes we added.
while len(expand_queue) > 0:
    parent = expand_queue.pop(0)
    # Ask node to sample its children.
    children = parent.sample_children()
    for child in children:
        # Always use default params for the decoration nodes.
        decorated_tree.add_node(child)
        decorated_tree.add_edge(parent, child)
        expand_queue.append(child)

In [37]:
#draw_scene_tree_contents_meshcat(decorated_tree, zmq_url=vis.window.zmq_url,  prefix="tree/contents")
#draw_scene_tree_structure_meshcat(decorated_tree, zmq_url=vis.window.zmq_url,  prefix="tree/structure",
#                                  with_triad=True, linewidth=30, node_sphere_size=0.02)

In [38]:
# Simulate forwards; everything but the new stuff should already be frozen.
projected_tree = project_tree_to_feasibility(decorated_tree, do_forward_sim=True)

In [39]:
draw_scene_tree_contents_meshcat(projected_tree, zmq_url=vis.window.zmq_url,  prefix="projected_tree/contents")
draw_scene_tree_structure_meshcat(projected_tree, zmq_url=vis.window.zmq_url,  prefix="projected_tree/structure",
                                  with_triad=True, linewidth=30, node_sphere_size=0.02)